In [100]:
#set notebook width to 90%
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Preprocess Text

#### Merge Abstract and Full text into single text file. For abstract part, it will be separated into sentences with `@highlight` token on top of each sentence.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [120]:
import numpy as np
import os
import logging
import pythainlp
from collections import deque
import sys
from tqdm import tqdm
from torch.utils.data import Dataset
sys.executable

'/Users/dai/anaconda3/bin/python'

#### To use sentence tokenizer, git clone from `dev` branch of pythainlp then install that repo instead.

Uninstall old one

In [15]:
!yes | pip uninstall pythainlp

Found existing installation: pythainlp 2.2.0.dev0
Uninstalling pythainlp-2.2.0.dev0:
  Would remove:
    /Users/dai/anaconda3/bin/thainlp
    /Users/dai/anaconda3/lib/python3.7/site-packages/pythainlp.egg-link
Proceed (y/n)?   Successfully uninstalled pythainlp-2.2.0.dev0
^C


Run the following command:  `git clone -b dev https://github.com/PyThaiNLP/pythainlp.git && cd ./pythainlp`

Install new one from `dev` branch

In [16]:
!pip install -e ../pythainlp

Obtaining file:///Users/dai/Desktop/pythainlp
  Running setup.py develop for pythainlp


### Example of Dataset used in benchmarking
- CNN/Dailymail

Here, you can see CNN/Dailymail dataset structure briefly.

In [15]:
def process_story(raw_story):
    """ Extract the story and summary from a story file.

    Arguments:
        raw_story (str): content of the story file as an utf-8 encoded string.

    Raises:
        IndexError: If the story is empty or contains no highlights.
    """
    nonempty_lines = list(filter(lambda x: len(x) != 0, [line.strip() for line in raw_story.split("\n")]))

    # for some unknown reason some lines miss a period, add it
    nonempty_lines = [_add_missing_period(line) for line in nonempty_lines]

    # gather article lines
    story_lines = []
    lines = deque(nonempty_lines)
    while True:
        try:
            element = lines.popleft()
            if element.startswith("@highlight"):
                break
            story_lines.append(element)
        except IndexError:
            # if "@highlight" is absent from the file we pop
            # all elements until there is None, raising an exception.
            return story_lines, []

    # gather summary lines
    summary_lines = list(filter(lambda t: not t.startswith("@highlight"), lines))

    return story_lines, summary_lines

def _add_missing_period(line):
    END_TOKENS = [".", "!", "?", "...", "'", "`", '"', "\u2019", "\u2019", ")"]
    if line.startswith("@highlight"):
        return line
    if line[-1] in END_TOKENS:
        return line
    return line + "."



In [16]:
filenames = os.listdir('./stories')
len(filenames), filenames[:10]

(16524,
 ['638ba1352bdf405a8f5bd681d7fe5c928686afff.story',
  'f9f9601180ab3278165d936821e8f145659997f3.story',
  '80ec0efb252ec4470aee44482d1e196111b5780b.story',
  '8435150be66ea9792999dfc233cc690f9c2fe2d0.story',
  '1444cf4d1832507a29a98529c2cd1a41f0154b52.story',
  'c302cdf2cafb85f08e0eb964f23736820b66a3ac.story',
  '95878b495e8b94f85e04e49ed84b2761d675f765.story',
  '4b8a784508a50f1fb8bcf799004af94cafe875b5.story',
  '88a754dc1a3f0cbce82bf7b8ec32f6bd62ac6d39.story',
  'd53cc805221115ab83e196f5b463f6586ea2cefe.story'])

In [17]:
documents = [] #store path to each text instead of loading into RAM
path='./stories'
for story_filename in filenames:
    if "summary" in story_filename:
        continue
    path_to_story = os.path.join(path, story_filename)
#     print(path_to_story)
    if not os.path.isfile(path_to_story):
        continue
    documents.append(path_to_story)
len(documents), documents[:5]

(16524,
 ['./stories/638ba1352bdf405a8f5bd681d7fe5c928686afff.story',
  './stories/f9f9601180ab3278165d936821e8f145659997f3.story',
  './stories/80ec0efb252ec4470aee44482d1e196111b5780b.story',
  './stories/8435150be66ea9792999dfc233cc690f9c2fe2d0.story',
  './stories/1444cf4d1832507a29a98529c2cd1a41f0154b52.story'])

### Text Example

In [22]:
document_path=documents[10]
raw_story=None
with open(document_path, encoding="utf-8") as source:
    raw_story = source.read()
print(raw_story)

HURUMA, Kenya (CNN)  -- We found Barack Obama's half-brother living in a Nairobi slum.

George Obama says he is sure his half-brother will win the U.S. presidency in November.

George Obama, whose birth certificate shows that he is Barack Obama's half-brother, lives in a small house in Huruma that he shares with his mother's extended family, far away from the presidential campaign circus.

In his memoir, "Dreams for my Father," the Democratic presidential candidate describes meeting George as a "painful affair." Barack Obama's trip to Kenya meant meeting family he had never known.

In the book, which is popular in Nairobi and can be found in almost any supermarket, Obama looks back at his personal story and his struggles to reconcile with a Kenyan father who left him and his mother when he was just a child.

Barack Obama Sr. died in a car accident when George was just 6 months old. And like his half-brother, George hardly knew his father. George was his father's last child and had not 

In [30]:
def show_story(idx):
    document_path = documents[idx]
    document_name = document_path.split("/")[-1]
    with open(document_path, encoding="utf-8") as source:
        raw_story = source.read()
        story_lines, summary_lines = process_story(raw_story)
    print('filename:',document_name)
    print('STORY')
    print('lines:',len(story_lines))
    print(story_lines)
    print('SUMMARY')
    print('lines:',len(summary_lines))
    print(summary_lines)
show_story(123)

filename: 32a7f2a0a2192434ac4da05c328117d2dc55c6ee.story
STORY
lines: 24
['Editor\'s note: Below is an excerpt from Larry King\'s autobiography, "My Remarkable Journey," published by Weinstein Books and available at bookstores nationwide.  Larry King anchors "Larry King Live at 9 p.m. ET on CNN.', 'Larry King recalls a much-needed win at the track during one of the lowest points of his life.', "I was thirty-seven years old. (In 1971). I had no job. I had a couple hundred thousand dollars in debts. And a four-year-old daughter. I'd take Chaia to our secret park on our visiting days. That's when the pain cut the deepest -- looking at my daughter and knowing I had no way to support her.", "Things got bleaker and bleaker. I became a recluse. By late May, I was down to forty-two dollars. My rent was paid only until the end of the month. I locked myself in my apartment wondering how bad things could possibly get. Pretty soon I wouldn't even be able to afford cigarettes. I remembered a night 

### Raw example of Thesis Dataset

In [23]:
raw_text = ''
with open('/Users/Dai/Downloads/drive-download-20200424T115746Z-001/acc60359kjp_ch1.pdf.txt', 'r') as f:
    raw_text = f.read()
raw_text

' \nบทที่ 1 \n \nบทนำ \n \n1.1 ประวัติควำมเป็นมำ \n \nสหกรณ์การเกษตรมีวัตถุประสงค์เพื่อช่วยเหลือความเป็นอยู่ของเกษตรกรที่เป็นสมาชิกให้ดีขึ้น โดยมีการดำเนินธุรกรรมในหลาย ๆ ด้าน ได้แก่ การให้สินเชื่อเพื่อการเกษตร การรับฝากเงิน การจัดหาสินค้าอุปโภคบริโภค วัสดุการเกษตร เครื่องมือ และอุปกรณ์การเกษตรมาจำหน่ายแก่สมาชิก การรวบรวมผลิตผลออกจำหน่าย การแปรรูปผลิตผลการเกษตร และการผลิตสินค้าเพื่อขาย ตลอดจนการส่งเสริมและให้บริการเพื่อการเกษตร การจัดระบบส่งน้ำการปรับพื้นที่เพื่อให้เกิดประโยชน์แก่การเพาะปลูกของสมาชิกยิ่งขึ้น ซึ่งสหกรณ์จะต้องบันทึกรายการอันเกิดจากธุรกรรมต่าง ๆ ให้ถูกต้องและเป็นปัจจุบัน อย่างไรก็ตามพบว่ามีสหกรณ์เป็นจำนวนมากที่ไม่สามารถจัดทำบัญชีและงบการเงินให้ถูกต้องและเป็นปัจจุบันได้ เนื่องจากความสามารถในการทำบัญชีของสหกรณ์การเกษตรยังค่อนข้างจำกัด จึงทำให้สหกรณ์ไม่สามารถใช้ประโยชน์จากข้อมูลทางการเงินการบัญชีเพื่อการบริหารงานได้ (กรมตรวจบัญชีสหกรณ์ 2554 : ออนไลน์) ซึ่งไม่สอดคล้องกับสภาวะในปัจจุบันที่โลกได้เข้าสู่ยุคข้อมูลข่าวสาร ซึ่งระบบสารสนเทศมีความสำคัญต่อการดำเนินงานของสหกรณ์ จึงมีคว

In [122]:
def contain_thai_character(sen):
    THAI_CHAR = pythainlp.thai_consonants+pythainlp.thai_vowels+'่้๋๊์'
    for c in sen:
        if c in THAI_CHAR:
            return True
    return False

def contain_digit(sen):
    DIGITS='0123456789'
    for c in sen:
        if c in DIGITS:
            return True
    return False

def all_is_digit(sen):
    DIGITS='0123456789'
    for c in sen:
        if c not in DIGITS:
            return False
    return True
    
def eliminate_continuous_whitespace(sent):
    a = []
    for i in range(len(sent)):
        if not (i > 0 and sent[i-1] == ' ' and sent[i] == ' '):
            a.append(sent[i])
    return ''.join(a)

def eliminate_brackets(sent):
    ''' Eliminate brackets from the sentence. Usually it is useless for summary, 
    because the citation was made in English, and we don't allow English in this 
    Dataset so it is gone anyway.
    '''
    con = 0
    a = []
    for i, c in enumerate(sent):
        if c == '(':
            con = 1
        if con == 0:
            a.append(c)
        if c == ')':
            con = 0
    return ''.join(a)


def clean(sen, mode='text'):
    s = pythainlp.thai_consonants+pythainlp.thai_vowels+pythainlp.thai_digits+'0123456789./=-+_-[]{};:,!@#$%^&*() '+'่้๋๊์'
    a = []
    for i, c in enumerate(sen):
        if c not in s:
            d = ''
        else:
            if i > 0 and sen[i-1] == ' ' and c == ' ':
                d = ''
            else:
                d = pythainlp.util.thai_digit_to_arabic_digit(c)
        a.append(d)
    new_sen = ''.join(a)
    
    #eliminate continuous whitespace
    new_sen = eliminate_continuous_whitespace(new_sen)
    
    #eliminate brackets
    new_sen = eliminate_brackets(new_sen)
    
    
    #shoud contain at least thai character or number, comma or other token only is not allowed
    if len(new_sen) == 1 and  not contain_thai_character(new_sen) and not contain_digit(new_sen):
        new_sen = ''
    elif not contain_thai_character(new_sen) and not all_is_digit(new_sen): 
        #if it does not contain ANY thai character, it should contain ONLY number
        new_sen = ''
        
    #eliminate sentence which is too short (still enough data)
    if len(new_sen) < 50 and mode != 'abs':
        new_sen = ''
        
    
    return new_sen

def clean_text(raw_text, mode='text'):
    return list(filter(lambda x: len(x) != 0,[clean(sent.strip(),mode=mode).strip() for sent in raw_text.split('\n')]))

def clean_and_tokenize_sentence(raw_text, mode='text'):
    """ Extract text -> clean text -> tokenize sentences -> list of sentences
    """
    cleaned_sentences = []
    sentences = clean_text(raw_text, mode=mode)
#     print(sentences)
    con=0
    if mode != 'abs': # not abstract
        con = 1
    for sent in sentences:
        if mode == 'abs' and sent == 'บทคัดย่อ':
            con=1
            continue
        if con:
            sts = pythainlp.sent_tokenize(sent, engine='crfcut')
            for st in sts:
                cleaned_sentences.append(st)

    return cleaned_sentences

def merge_dataset(path):
    ''' This function merge dataset into proper format
    (as close to CNN/Dailymail format as possible)
    '''
    
    filepaths = []
    for folder in os.listdir(path):
        if folder[0] != '.':
            filepaths += [os.path.join(path, folder, fnme) for fnme in os.listdir(os.path.join(path, folder)) if fnme[0] != '.']
    print('There are total of {} files'.format(len(filepaths)))
    cache = {}
    text_idx = 0
    text_cache = []
    #iterate every file to group dataset into filenames
    for i, filepath in tqdm(enumerate(filepaths), position=0, leave=True):
        filename = filepath.split('/')[-1]
        if filename[0] == '.':
            continue
        raw_text = ''
        with open(filepath, 'r') as f:
            raw_text = f.read()
        try:
            st = filename.split('.')[0]
            filetype = st[-3:]
            filename = st[:-4]
        except:
            raise ValueError('error: {}'.format(filename))
        #check if filename is registered to cache
        if filename not in cache:
            #initialize
            cache[filename] = {'abs': None, 'text': []}
            
        #save raw text to cache
        if filetype == 'abs': #abstract
            cache[filename]['abs'] = text_idx
            text_cache.append(raw_text)
            text_idx += 1
        else: #text
            try:
                cache[filename]['text'].append(text_idx)
            except:
                cache[filename]['text'] = [text_idx]
            text_cache.append(raw_text)
            text_idx += 1
#     print(cache)
    final_text = {}
    for i, item in tqdm(enumerate(cache.keys()), position=0, leave=True):
        text_full = ''
        #preprocess full text
        text_sentences = '\n\n'.join(clean_and_tokenize_sentence('\n'.join([text_cache[rt] for rt in cache[item]['text']]), mode='text'))
            
        # preprocess abstract
        try:
            abs_sentences = '\n\n@highlight\n\n'.join(clean_and_tokenize_sentence(text_cache[cache[item]['abs']], mode='abs'))
        except:
            #If for any text's abstract is missing, we will skip that text
            logging.critical('WARNING: Abstract of {} does not exist'.format(item))
            continue
        
        #merge full text and abstract
        final_text[item] = text_sentences + abs_sentences
        
    return cache, final_text



cache, final_text = merge_dataset(path='/Users/Dai/Downloads/cmu_papers')

316it [00:00, 1563.01it/s]

There are total of 1654 files


1654it [00:01, 1475.60it/s]
35it [00:52,  1.31s/it]CRITICAL:root:WARNING: Abstract of mba61159knw does not exist
99it [02:52,  1.92s/it]CRITICAL:root:WARNING: Abstract of midw60859npk does not exist
101it [02:56,  1.95s/it]CRITICAL:root:WARNING: Abstract of acc60759bjs does not exist
102it [02:58,  1.92s/it]CRITICAL:root:WARNING: Abstract of mpa60359byl does not exist
104it [03:05,  2.65s/it]CRITICAL:root:WARNING: Abstract of agbus61259nyk does not exist
108it [03:11,  1.78s/it]CRITICAL:root:WARNING: Abstract of encm60759nsn does not exist
110it [03:15,  1.81s/it]CRITICAL:root:WARNING: Abstract of econ60560ysr does not exist
114it [03:24,  2.15s/it]CRITICAL:root:WARNING: Abstract of mph60859tsh does not exist
119it [03:33,  2.43s/it]CRITICAL:root:WARNING: Abstract of mpa61059bks does not exist
125it [03:44,  2.01s/it]CRITICAL:root:WARNING: Abstract of midw61159tds does not exist
132it [03:51,  1.24s/it]CRITICAL:root:WARNING: Abstract of coms61259plt does not exist
133it [03:53,  1.37s/

In [123]:
cache

{'nuadm60659tpt': {'abs': 242, 'text': [0, 120, 226, 355, 392]},
 'mph60559wwc': {'abs': 211, 'text': [1, 116, 194, 358, 386]},
 'nuad60759jpw': {'abs': 438, 'text': [2, 115, 385, 459]},
 'patho60459tjt': {'abs': 17, 'text': [3, 195, 230, 460, 582]},
 'nuic60659kpy': {'abs': 16, 'text': [4, 196, 229, 457, 583]},
 'nuped60859kny': {'abs': 440, 'text': [5, 118, 359, 388, 455]},
 'acma60459kmk': {'abs': 6, 'text': [213, 249, 442, 568]},
 'acc60959tud': {'abs': 22, 'text': [7, 189, 219, 469, 585]},
 'km60559kit': {'abs': 21, 'text': [8, 192, 218, 468, 587]},
 'chem60916kpj': {'abs': 208, 'text': [9, 127, 188, 383]},
 'hort60759tww': {'abs': 25, 'text': [10, 187, 351, 466, 591]},
 'acc60959ksw': {'abs': 240, 'text': [11, 123, 221, 352, 380]},
 'enso61258oks': {'abs': 461, 'text': [12, 134, 343, 375, 446]},
 'acc61158skw': {'abs': 579, 'text': [13, 135, 241, 344, 377, 562]},
 'nuad61258tcw': {'abs': 228, 'text': [14, 129, 247, 346, 370]},
 'enel60459kdh': {'abs': 584, 'text': [15, 133, 245, 

Data is too Longggggggggg TT

In [124]:
for i, item in enumerate(cache.keys()):
    if i > 0:
        break
    print(item)
    print(final_text[item])
    

nuadm60659tpt
การวิจัยครั้งนี้เป็นการวิจัยเชิงพรรณนาแบบหาความสัมพันธ์  

โดยมีวัตถุประสงค์เพื่อศึกษาความสัมพันธ์ระหว่างคุณภาพการพยาบาลกับเอกสิทธิ์วิชาชีพและบรรยากาศของทีม โดยมีขั้นตอนการวิจัย ดังนี้

ประชากรที่ใช้ในการวิจัยครั้งนี้ เป็นพยาบาลวิชาชีพที่ปฏิบัติงานอยู่ในแผนกผู้ป่วยในของโรงพยาบาลเครือมูลนิธิสภาคริสตจักรแห่งประเทศไทย 3 แห่ง คือโรงพยาบาลแมคคอร์มิค โรงพยาบาลโอเวอร์บรุ๊ค และโรงพยาบาลกรุงเทพคริสเตียน รวมทั้งหมด 370 คน จากสถิติบุคลากรฝ่ายการพยาบาลของแต่ละโรงพยาบาล ณ วันที่ 30 มีนาคม พ.ศ. 2558

กลุ่มตัวอย่างประกอบด้วยพยาบาลวิชาชีพ ที่ปฏิบัติงานในโรงพยาบาลกรุงเทพคริสเตียน โรงพยาบาลโอเวอร์บรุ๊ค และโรงพยาบาลแมคคอร์มิค เป็นเวลาอย่างน้อย 1 ปี ไม่ดำรงตำแหน่งบริหารและยินดีให้ความร่วมมือในการตอบแบบสอบถาม ผู้วิจัยได้คำนวณขนาดของกลุ่มตัวอย่างโดยใช้สูตรของทาโรยามาเน่  กำหนดความคลาดเคลื่อนที่ระดับ 0.05  ได้ขนาดกลุ่มตัวอย่าง 192 คน

เนื่องจากการวิจัยครั้งนี้เป็นการเก็บรวบรวมข้อมูล โดยใช้แบบสอบถามและส่งให้กลุ่มตัวอย่างทางไปรษณีย์ ซึ่งอาจได้รับคืนไม่ครบ ไม่สมบูรณ์ หรือมีโอกาสที่แบบสอบถามจะสูญหา

In [125]:
path='/Users/Dai/Downloads/cmu_papers_merged/'
for item in final_text.keys():
    with open(os.path.join(path, item+'.story'), 'w') as f:
        f.write(final_text[item])

In [39]:
path='/Users/Dai/Downloads/cmu_papers'
filepaths = []
for folder in os.listdir(path):
    if folder[0] != '.':
        filepaths += [os.path.join(path, folder, fnme) for fnme in os.listdir(os.path.join(path, folder)) if fnme[0] != '.']
len(filepaths)

1654

In [40]:
filepaths[:10]

['/Users/Dai/Downloads/cmu_papers/35201-35300/nuadm60659tpt_ch3.pdf.txt',
 '/Users/Dai/Downloads/cmu_papers/35201-35300/mph60559wwc_ch2.pdf.txt',
 '/Users/Dai/Downloads/cmu_papers/35201-35300/nuad60759jpw_ch5.pdf.txt',
 '/Users/Dai/Downloads/cmu_papers/35201-35300/patho60459tjt_ch1.pdf.txt',
 '/Users/Dai/Downloads/cmu_papers/35201-35300/nuic60659kpy_ch1.pdf.txt',
 '/Users/Dai/Downloads/cmu_papers/35201-35300/nuped60859kny_ch5.pdf.txt',
 '/Users/Dai/Downloads/cmu_papers/35201-35300/acma60459kmk_abs.pdf.txt',
 '/Users/Dai/Downloads/cmu_papers/35201-35300/acc60959tud_ch1.pdf.txt',
 '/Users/Dai/Downloads/cmu_papers/35201-35300/km60559kit_ch1.pdf.txt',
 '/Users/Dai/Downloads/cmu_papers/35201-35300/chem60916kpj_ch2.pdf.txt']

In [31]:
class ThesisDataset(Dataset):
    """ Load our Thai Thesis Summary Dataset.

    The class will process the documents that are located in the specified
    folder.
    """

    def __init__(self, path="", prefix="train"):
        """ We initialize the class by listing all the documents to summarize.
        Files are not read in memory due to the size of some datasets (like CNN/DailyMail).
        """
        assert os.path.isdir(path)

        self.documents = []
        story_filenames_list = os.listdir(path)
        for story_filename in story_filenames_list:
            path_to_story = os.path.join(path, story_filename)
            if not os.path.isfile(path_to_story):
                continue
            self.documents.append(path_to_story)

    def __len__(self):
        """ Returns the number of documents. """
        return len(self.documents)

    def __getitem__(self, idx):
        
        document_path = self.documents[idx]
        document_name = document_path.split("/")[-1]
        with open(document_path, encoding="utf-8") as source:
            raw_story = source.read()
            story_lines, summary_lines = process_story(raw_story)
        return document_name, story_lines, summary_lines
    
dts = ThesisDataset(path='/Users/Dai/Downloads/drive-download-20200424T115746Z-001')
len(dts)

6